<a href="https://colab.research.google.com/github/kimbaorrr/colab_exercises/blob/main/Other%20Notebooks/Satpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install libgeos-dev htop p7zip-full -y

!pip install "satpy[ahi_hsd, geotiff, rayleigh, overlays, tests]==0.46.0" himawari_api gofile2

In [ ]:
from satpy import Scene
from pyresample import create_area_def
import pandas as pd
import json
from datetime import datetime
import himawari_api
from requests import request
import sys
from gofile2 import Gofile
from glob import glob
import os


In [ ]:
!wget -nc "https://github.com/kimbaorrr/cdn/releases/download/CDN/satpy.zip"
!wget -nc "https://www.ngdc.noaa.gov/mgg/shorelines/data/gshhg/oldversions/version2.3.6/gshhg-shp-2.3.6.zip"

!7z x ./satpy.zip -aos -y
!7z x ./gshhg-shp-2.3.6.zip -aos -y

print('Xong !!!')

In [ ]:
!cp -rf  ./satpy /usr/local/lib/python{sys.version[:4]}/dist-packages

!cp -rn ./WDBII_shp /tmp
!cp -rn ./GSHHS_shp /tmp

print('Xong !!!')

In [ ]:
# Tải dữ liệu từ tệp JSON
response = request("GET", "https://himawari8-dl.nict.go.jp/himawari8/img/D531106/latest.json")
# Đọc dữ liệu & Convert date
data_json = json.loads(response.text)
today = pd.to_datetime(data_json['date'])

year = today.year
month = today.month
day = today.day
hour = today.hour
minute = today.minute

In [ ]:
start_time = datetime(year, month, day, hour, minute)
end_time = start_time
print(f'{start_time} - {end_time}')

filter_parameters = {}
filter_parameters["channels"] = None
filter_parameters["scene_abbr"] = None

filenames = himawari_api.download_files(
    base_dir="/tmp/",
    protocol="s3",
    fs_args={},
    satellite="HIMAWARI-9",
    product_level="L1B",
    product="Rad",
    sector="FLDK",
    start_time=start_time,
    end_time=end_time,
    filter_parameters=filter_parameters,
    n_threads=16,
    force_download=False,
    check_data_integrity=True,
    progress_bar=True,
    verbose=True
)

print('Xong !!!')

In [ ]:
# Thiết lập chung
choose_product = 'geo_color'
img_save_path = './'
save_name = f'Himawari_9_{choose_product}_{today.strftime("%Y%m%d%H%M%S")}.tif'

# Bắt đầu Render
print('Nạp dữ liệu')
scn = Scene(reader='ahi_hsd', filenames=filenames)
scn.load([choose_product])

print('Lấy mẫu lại')
# area_def = create_area_def('Vietnam',
#                            {'proj': 'longlat', 'datum': 'WGS84'},
#                            area_extent=(100, 7, 118, 25),
#                            units='degrees',
#                            shape=(2000, 2000)
#                           )

area_def = scn['B03'].attrs['area']
new_scn = scn.resample(area_def)

print('Xuất file TIF')
new_scn.save_dataset(choose_product,
                     base_dir=img_save_path,
                     filename=save_name,
                     writer='geotiff',
                     num_threads=16,
                     overlay={'coast_dir': '/tmp/', 'color': (255, 255, 255), 'width': 1, 'level_coast': 1, 'level_borders': 2}
                     )

print('Xong !!!')
print(f'=> Tên tệp đã xuất: {save_name}')

# Upload lên Gofile

In [ ]:
# Lấy file tif mới nhất

list_of_files = glob('*.tif')
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

In [ ]:
g_a = await Gofile.initialize('l64xORIRqPbDG8IsYhr8swQY536kwHmq')
await g_a.upload(latest_file)
await g_a.done()

print('Xong !!!')

Xong !!!
